<a href="https://colab.research.google.com/github/Frank-III/Fin553-ML-in-Finance/blob/main/553_Project_2_LSMC_with_JAX_my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Due date: October 21 2022 

# Description
  In this problem we will use apply the LSMC method to price American put options. Specifically, we will replicate the result in the first row, 6th column of Table 1 in [Longstaff and Schwartz 2001](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf)

  

*  Read the introduction of the [paper](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf).
*   We will price an american put option as described in page 126 of the aforementioned article. Read paragraphs 1 and 2 of page 126
* As we saw in class, one of the ways we can use linear regression to fit nonlinear functions is to use polynomial features. A common choice in many applications is to use the so called ``Chebyshev polynomials''. Chebyshev polynomials are defined recursively by:

\begin{equation}
T_0(x) = 1\\
T_1(x) = x\\
T_{n + 1}(x)  =  2 x T_n(x) - T_{n - 1}(x)\\
\end{equation}


# Part 1
The code below simulates the evolution of a stock price that follows a geometric brownian motion. Write a JAX version of that code. You are not allowed to use functions from other libraries. For this part, the "simulate"
function does not need to be jit compiled. As we will see, jit compiling a funciton with for loops may introduce some complications
 

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100


def simulate():
  #np.random.seed(0)
  key = jax.random.PRNGKey(1)

  @jax.jit
  def step(S):
    dZ = jax.random.normal(key, S.shape) * jnp.sqrt(dt)
    dS = r * S  * dt + σ  * S  * dZ
    S = S + dS
    return S

  S0 = jnp.ones(20000)
  S = S0
  S_list = []

  for t in range(m):
    S = step(S)
    S_list.append(S)

  S_array = jnp.stack(S_list)
  return S_array

simulate()

DeviceArray([[0.99656045, 0.997612  , 0.9988988 , ..., 0.995853  ,
              0.9948579 , 0.9989114 ],
             [0.9931327 , 0.9952297 , 0.99779886, ..., 0.9917232 ,
              0.9897423 , 0.9978239 ],
             [0.98971677, 0.9928531 , 0.9967001 , ..., 0.9876105 ,
              0.98465294, 0.9967376 ],
             ...,
             [0.7134402 , 0.7911217 , 0.89764947, ..., 0.66547644,
              0.6033702 , 0.89875424],
             [0.71098626, 0.7892325 , 0.896661  , ..., 0.6627167 ,
              0.6002676 , 0.8977758 ],
             [0.7085408 , 0.7873478 , 0.8956736 , ..., 0.6599684 ,
              0.59718096, 0.8967985 ]], dtype=float32)

# Part 2
Write a jit compiled version of the simulate function. You may want to check out the function jax.lax.scan.


In [ ]:
from pickle import NONE
@jax.jit
def simulate_jax():
  key = jax.random.PRNGKey(1)
  def step(init, S):
    key, S = init
    key, subkey = jax.random.split(key)
    dZ = jax.random.normal(subkey, S.shape) * jnp.sqrt(dt)
    dS = r * S  * dt + σ  * S  * dZ
    S = S + dS
    return (key,S), S

  S0 = jnp.ones(20000)
  S = S0
  m = 100

  return jax.lax.scan(step, (key,S),xs = None, length = m)

simulate_jax()

((DeviceArray([3723834036,    5122834], dtype=uint32),
  DeviceArray([1.0296268 , 1.0981548 , 1.0153092 , ..., 1.0037547 ,
               1.0099921 , 0.98250884], dtype=float32)),
 DeviceArray([[1.0017602 , 0.9997457 , 1.0046163 , ..., 0.9960504 ,
               0.9964467 , 1.0037811 ],
              [0.99657446, 0.99958307, 1.0009022 , ..., 0.9956731 ,
               0.99473   , 1.0082498 ],
              [1.0010644 , 1.0030092 , 1.0055163 , ..., 0.9962548 ,
               0.99425787, 1.0071009 ],
              ...,
              [1.0310631 , 1.0890951 , 1.0143472 , ..., 0.9992722 ,
               1.0093237 , 0.9828975 ],
              [1.031782  , 1.0884858 , 1.0134048 , ..., 1.0009794 ,
               1.009332  , 0.98655933],
              [1.0296268 , 1.0981548 , 1.0153092 , ..., 1.0037547 ,
               1.0099921 , 0.98250884]], dtype=float32))

# Part 3
The code below computes the price of an American Put option using Least Squares Monte Carlo (LSMC). Write a JAX version of that code. You are not allowed to use functions from other libraries. Your "compute_price" function must be jit compiled.

In [ ]:
#Jax version
import numpy as np

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
Δt = T / m  # interval between two exercise dates


# Construct polynomial features of order up to k using the
# recursive formulation

def chebyshev_basis(x_me, k):
  def func(carry, x):
    last_1, last_2 = carry
    new = 2 * x_me * last_1 - last_2
    return (new, last_1), new
  return jax.lax.scan(func, (x_me, jnp.ones(len(x_me))) , xs=None, length=k-2)

# scales x to be in the interval(-1, 1)

def scale(x):
    xmin = x.min()
    xmax = x.max()
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b


# simulates one step of the stock price evolution

def step(init, S):
  key, S = init
  key, subkey = jax.random.split(key)
  dB = jnp.sqrt(Δt) * jax.random.normal(subkey, S.shape)
  S += r * S * Δt + σ * S * dB
  return (key, S), S


def payoff_put(S):
    return jnp.maximum(K - S, 0.)

# LSMC algorithm
def compute_price():
  key = jax.random.PRNGKey(1)
  S0 = Spot * jnp.ones(n)

  S = jax.lax.scan(step, (key, S0), xs=None, length=m)[1]
  discount = jnp.exp(-r * Δt)

  value_if_exercise = payoff_put(S[-1])
  discounted_future_cashflows = value_if_exercise * discount

  def new_func(init, x):
    X = chebyshev_basis(scale(S[-2 - x]), order)[1].T
    Θ = jnp.linalg.solve(X.T @ X, X.T @ init)
    value_if_wait = X @ Θ
    value_if_exercise = payoff_put(S[-2 - x])
    exercise = value_if_exercise >= value_if_wait
    Y = discount * jnp.where(exercise, value_if_exercise, init)
    return Y, Y

  return jax.lax.scan(new_func, discounted_future_cashflows, xs = jnp.array(range(m-1)))[0].mean()


compute_price()
#test = compute_price(order, Spot, σ, K, r)

DeviceArray(4.466024, dtype=float32)